# Clustering Practice

In this notebook you'll use the techniques presented in Notebook 1 - Clustering to work through two clustering problems.

In particular you'll work through two problems presented in the following resources:
<ol>
    <li><a href="https://www.amazon.com/Hands-Unsupervised-Learning-Using-Python-ebook/dp/B07NY447H8/ref=sr_1_1?crid=2ENKBD4O6EPNX&dchild=1&keywords=hands+on+unsupervised+learning&qid=1589566161&sprefix=hands+on+un%2Caps%2C170&sr=8-1">Hands-On Unsupervised Learning Using Python</a>, Chapter 6, and</li>
    <li><a href="https://www.amazon.com/Basketball-Data-Science-Applications-Chapman-ebook/dp/B083G6PQV2/ref=sr_1_1?crid=3D5B29MXA6E3C&dchild=1&keywords=basketball+data+science+with+applications+in+r&qid=1589566208&sprefix=basketball+data+scie%2Caps%2C172&sr=8-1">Basketball Data Science With Applications in R</a></li>.
</ol>

## What You'll Accomplish

In particular you'll work on:
<ul>
    <li>clustering loan applications to help identify good investments,</li>
    <li>using clustering to identify basketball players that play similar styles.</li>
</ul>

In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

## Lending Club Data

Below we load in and do some preprocessing on publicly available data from <a href="https://www.lendingclub.com/">Lending Club</a>, a US peer-to-peer lending company. At the time this data was collected borrowers could request a loan from $\$1,000$ to $\$40,000$. 

Each row of the data set represents a funded loan from the platform. The columns have information about the loan itself and the person that applied for the loan. 

A loan `grade` column is also included and generally implies that this loan was good from the lenders point of view.

One type of problem you may want to solve is to build a model that classifies a loan's grade based on the other features in the data set.

However, another interesting problem would be to cluster observations and look for patterns in the data. Not just for the purposes of identifying highly graded loans, but also for identifying non obvious lending patterns.

### Cleaning Data

I first walk through cleaning the data a little bit, then I leave it to you to explore.

In [ ]:
# This reads in the raw csv
# from a github repo
# don't worry that warning is supposed to happen
lending = pd.read_csv("https://raw.githubusercontent.com/aapatel09/handson-unsupervised-learning/master/datasets/lending_club_data/LoanStats3a.csv")

print("The data has", len(lending), "observations.")

In [ ]:
# We drop a bunch of empty columns
columns_to_keep = ['grade','sub_grade','loan_amnt','funded_amnt',
                 'funded_amnt_inv','term', 'int_rate','installment', 
                 'annual_inc', 'dti','delinq_2yrs','open_acc','pub_rec','revol_bal',
                 'revol_util','total_acc','out_prncp', 
                 'out_prncp_inv','total_pymnt','total_pymnt_inv', 
                 'total_rec_prncp','total_rec_int','total_rec_late_fee', 
                 'recoveries','collection_recovery_fee','last_pymnt_amnt']

In [ ]:
# I now keep those columns
lending = lending.loc[:,columns_to_keep]

In [ ]:
# let's examine the first 5 rows
lending.head()

In [ ]:
# These three columns are strings but should be numbers
lending[['term','int_rate','revol_util']]

In [ ]:
lending.loc[lending.term.notna(),'term'] = lending['term'].dropna().apply(lambda x: int(x.strip().split(" ")[0]))

In [ ]:
lending.loc[lending.int_rate.notna(),'int_rate'] = lending['int_rate'].dropna().apply(lambda x: float(x.strip().split("%")[0]))

In [ ]:
lending.loc[lending.int_rate.notna(),'revol_util'] = lending['revol_util'].dropna().apply(lambda x: float(x.strip().split("%")[0]))

In [ ]:
# Much Better
lending[['term','int_rate','revol_util']]

In [ ]:
# let's impute those nan values with the median
for c in columns_to_keep[2:]:
    replace = lending[c].median()
    
    lending.loc[lending[c].isna(),c] = replace

In [ ]:
lending

### Ready to Go

Now that the data is clean let's go ahead and apply clustering.

Feel free to apply any of the clustering techniques we've used so far. You may want to compare the output to the `grade` and `sub_grade` columns.

The goal here is to:
<ol>
    <li>get more familiar with the clustering techniques we learned in notebook 1, and</li>
    <li>see if you can gain any insight about the lending data set.</li>
</ol>

Remember that you'll need to scale the data!

In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







## Shooters Gotta Shoot

These clustering techniques are often applied in sports analytics.

They can be helpful in finding commonalities between players and teams in a number of ways.

In this problem you'll load in some shot distributions for NBA players from the 2018-19 NBA season.

### The Data

Each row of the data set will be a different player with their percentage of shots taken in different regions of the basketball court along with what percentage of those shots they made. 

Here is a basketball court cut into 15 regions,
<img src="CourtZones.png" style="width:50%;"></img>

Let's load in the data and look at it.

In [ ]:
nba = pd.read_csv("players_18_19.csv")

In [ ]:
nba.head()

Explore this data set and perform clustering on either the whole data set or just selected columns. Are you able to identify different shooting styles?

In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







In [ ]:
## Code here







See you in notebook 3!